In [1]:
import numpy as np
import pandas as pd

In [2]:
def roll_dice():
    return(np.random.randint(1,7))

def shuffle(df):
    return(df.sample(frac=1).reset_index(drop=True))

def take_cards(df, df_total, n=1):
    if (df.shape[0] == 0):
        df = shuffle(df_total)
    card = df.head(n)
    df.drop(index=df.index[:n], axis=0, inplace=True)
    return(df, card)

In [ ]:
class Deck(DataFrame):
    def shuffle(self):
        self.sample(frac=1).reset_index(drop=True)

In [3]:
class Battle:
    def __init__(self, player, monster):
        self.monster = monster
        self.player = player
    def escape_before(self):
        check = [i for i in self.monster.before_battle
                 if i.startswith('self.monster.kill')]
        if len(check)!=0:
            self.monster.kill()
    def fight(self):
        if (self.player.bonus > self.monster.level):
            self.get_reward()
        else:
            while (self.player.decision['one_hot'] > self.player.coef['one_hot'] 
                   and '<there are one hots in the hand>'):
                # select what cards and how many to use?
                # how to actualize????
                self.player.decision['one_hot'] -= (self.player.decision['one_hot'] *
                                                    self.player.learning_rate['one_hot'])
                # how to actualize????
                if (self.player.bonus > self.monster.level):
                    self.get_reward()
                    break
            if (self.player.bonus < self.monster.level):        
                self.escape_after()
    
    def escape_after(self, n=roll_dice()):
        #self.monster.even_if_escaped() -- evaluate
        if n+self.player.escape < self.monster.escape:
            self.get_bad_stuff()
    def get_reward(self):
        self.player.update_level(+monster.reward_level)
        self.player.update_hand(n=monster.reward_treasure, h_type='treasures',
                                new=take_cards(df, df_total, n=monster.reward_treasure))
        # !!!! somehow pass card decks + remove from deck -- maybe create another class:)
    def get_bad_stuff(self):
        for i in self.monster.bad_stuff:
            eval(i)

In [4]:
class Monster:
    def __init__(self, df):
        self.name = df.name[0]
        self.level = df.level[0]
        self.undead = df.undead[0]
        self.before_battle = df.before_battle[0].split(';')
        self.even_if_escaped = df.even_if_escaped[0].split(';')
        if df.escapable[0]:
            self.escape = 4
        else:
            self.escape = 10e7
        self.bad_stuff = df.bad_stuff[0].split(';')
        self.reward_level = df.reward_level[0]
        self.reward_treasure = df.reward_treasure[0]
        self.multiplier = 1
    def update_level(self, n, condition=None, update_type=None):
        if update_type == 'monster_buff' and self.multiplier > 1:
            n *= self.multiplier
        if condition is None:
            self.level += n
        elif eval(condition):
            self.level += n
    def kill(self, condition=None, treasures=True, levels=True):
        if condition is None:
            self.level = -10e7
        elif eval(condition):
            self.level = -10e7
            
        if not treasures:
            self.reward_treasure = 0
        if not levels:
            self.reward_level = 0
    def update_escape(n, condition=None):
        if condition is None:
            self.escape += n
        elif eval(condition):
            self.escape += n
    def update_multiplier(n):
        self.multiplier += n
    def update_treasures(n, condition=None):
        if condition is None:
            self.reward_treasure += n
        elif eval(condition):
            self.reward_treasure += n
        
    def update_reward_level(n, condition=None):
        if condition is None:
            self.reward_level += n
        elif eval(condition):
            self.reward_level += n

In [5]:
class Munchkin:
    def __init__(self, name='John Doe', sex='female', 
                 level=1, pclass=[], race=['human'],
                 hand=dict(doors=pd.DataFrame(),
                           treasures=pd.DataFrame(
                               dict(name=[], eq_type=[], big=[], restriction=[],
                                    bonus=[], hands=[], buffs=[], price=[]))),
                 equiped=pd.DataFrame(
                     dict(name=[], eq_type=[], big=[], restriction=[],
                          bonus=[], hands=[], buffs=[], price=[]))
                ):
        self.decisions = dict(fight=1, one_hot=1, auto_escape=1)
        self.coefs = dict(fight=0.5, one_hot=0.5, auto_escape=0.1)
        self.learning_rate = dict(fight=0.005, one_hot=0.005, auto_escape=0.005)
        self.name = name
        self.sex = sex
        self.pclass = pclass
        self.race = race
        self.level = level
        self.escape = 0
        self.bonus = level
        self.equiped = equiped
        self.hand = hand
        self.limits = dict(armor=1, boots=1, head=1, hands=2, big=1, cards=5)
    def update_level(self, n, condition=None):
        if condition is None:
            self.level += n
        elif eval(condition):
            self.level += n
        
        if self.level < 1:
            self.level = 1
    def update_pclass_race(self, what, make_decision=False, n=1, which=0):
        # which could be index selected by model or particular class/list of classes
        if n == -1 and select_yourself:
            eval('self.'+what+'.pop(self.'+what+'.index(which))')
        elif n == -1 and not(select_yourself):
            eval('self.'+what+'.pop(which)')
        elif n == -2:
            eval('self.'+what+'.clear()')
        elif n > 0:
            eval('self.'+what+'.extend(which)')
            
        if what=='race' and len(self.race) == 0:
            self.race = ['human']
    def update_sex(self, change=False):
        if change == False:
            if self.sex == 'male':
                self.sex = 'female'
            else:
                self.sex = 'male'
        else:
            self.sex = []
    def update_limit(self, which, n):
        self.limits[which] += n
    def update_hand(self, n=None, condition=None, everything=False, 
                    make_decision=True, new=None):
        # treasures, doors, any
        
        # how many is determined by which (list of indices)
        # can be a player's decision or other players' decision
        if n > 0:
            self.hand[h_type] = pd.concat([self.hand[h_type], new]).reset_index(drop=True)
        else:
            if condition is not None:
                self.hand[h_type] = self.hand[h_type].loc[eval(condition), :]
            elif everything:
                if h_type == 'treasures':
                    self.hand[h_type] = pd.DataFrame(
                        dict(name=[], eq_type=[], big=[], restriction=[],
                             bonus=[], hands=[], buffs=[], price=[])
                    )
                else:
                    self.hand[h_type] = pd.DataFrame(
                        dict()
                    )
            else:
                self.hand[h_type] = self.hand[h_type].iloc[which, :]
    def lose_equiped(self, condition=None, everything=False, make_decision=True):
        # how many is determined by which (list of indices)
        # can be a player's decision or other players' decision
        if condition is not None:
            self.equiped = self.equiped.loc[eval(condition), :]
        elif everything:
            self.equiped = pd.DataFrame(
                dict(name=[], eq_type=[], big=[], restriction=[],
                     bonus=[], hands=[], buffs=[], price=[])
            )
        else:
            self.equiped = self.equiped.iloc[which, :]
    def equip(self, new_eq=None):
        pass
    def use_one_shot(self):
        pass
    def sell(self):
        pass
    def update_temp_bonus(self):
        pass
    def die(self, marauder=True):
        pass
    def give_away(self):
        pass
    def make_decision(self):
        pass

In [6]:
monsters = pd.read_excel('example.xlsx',sheet_name=0)
races = pd.read_excel('example.xlsx',sheet_name=2)

dect_doors = pd.concat([monsters, races]).reset_index(drop=True)
deck_treasures = pd.read_excel('example.xlsx',sheet_name=1)

dect_doors = shuffle(dect_doors)
deck_treasures = shuffle(deck_treasures)

dect_doors_current = dect_doors.copy()
deck_treasures_current = deck_treasures.copy()

player_1 = Munchkin()

In [1]:
a = "self.monster.update_level(+5, condition='\'elf\'  in self.player.race');self.monster.update_level(+5, condition='\'halfling\' in self.player.race');self.monster.update_level(+5, condition='\'bard\' in self.player.plcass')"

In [3]:
a

["self.monster.update_level(+5, condition=''elf'  in self.player.race')",
 "self.monster.update_level(+5, condition=''halfling' in self.player.race')",
 "self.monster.update_level(+5, condition=''bard' in self.player.plcass')"]

In [ ]:
# Set queue:
current_player = player_1
fight_coef = 0.5

# Prepare:
# sell
# equip

# Main game:
dect_doors_current, door_card = take_cards(dect_doors_current, dect_doors, 1)
if door_card['type'].tolist()[0] == 'monster':
    print('Battle begins..')
    # current_monster = Monster(door_card)
    # current_player = Battle(current_player, current_monster).escape_before().fight().player
elif door_card['type'].tolist()[0] == 'curse':
    print('Curse..')
    #current_player.curse(door_card)
# if bozestwennoye wmeszatelstwo
else:
    print('Taking door card..')
    current_player.update_hand(n=1, h_type='doors', new=door_card)
    dect_doors.drop(dect_doors[dect_doors.name!=door_card.name[0]].index, inplace=True)
    if (current_player.decisions['fight'] > fight_coef and 
    'monster' in current_player.hand['doors']['type'].tolist()):
        print('Battle begins..')
        current_player = battle(current_player, 'which to fight???') # !!!!!!!!!!!!!!!!!!!!
    else:
        print('Taking door card..')
        dect_doors_current, door_card = take_cards(dect_doors_current, dect_doors, 1)
        current_player.update_hand(n=1, h_type='doors', new=door_card)
        dect_doors.drop(dect_doors[dect_doors.name!=door_card.name[0]].index, inplace=True)
        
# Finalize:
# Update decisions:
# Check convergence:

In [17]:
n = 2
n =3
n

SyntaxError: can't use starred expression here (Temp/ipykernel_10152/1265840399.py, line 2)

In [9]:
current_player = player_1
dect_doors_current, door_card = take_cards(dect_doors_current, dect_doors, 1)


In [13]:
current_monster = Monster(door_card)
print(current_monster.level)
current_monster.update_level(5, 'current_player.level == 1')
print(current_monster.level)

3.0
8.0


In [15]:
'race==\'halfling\''

"race=='halfling'"

In [ ]:

            

def battle(player, monster):
    
    kill_result = 
    
        
    if (kill_result == 'not killed' or len(check)==0):
        for i in monster.before_battle:
            eval(i)
        if player.bonus > monster.level[0]:
            kill_result = 'killed'
        else:
            while player.decision['use_one_hot'] and 'amount of one hot >0':
                # use one_hot
                # update coefs
                # update reward
    
    if (kill_result == 'not killed'):
        # try escape
        # if was not successfull
        # for i in monster.bad_stuff:
           # eval(i)
    
    # finally, take cards or suffer bitch :)
    # update 
    # update reward
    return player

In [18]:
from datetime import datetime
datetime.now().weekday
# 4 - pt
# 5 - sb
# 7 - ws

3

In [ ]:
# simplification:
# only 1 player
# no distinguish between open /close

# drop class/race every single moment!!!! use cards any single momet - make decision!